In [1]:
from fastai.vision import *
from warnings import filterwarnings
# from io import BytesIO
import numpy as np

from PIL import Image
import torch
from torch.autograd import Variable
from torchvision import transforms
from annoy import AnnoyIndex
from PIL import ImageFile

In [2]:
filterwarnings('ignore')
scaler = transforms.Resize((512, 512))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

In [3]:
sz = 512
emb_size = 512
data = ImageDataBunch.single_from_classes('../models/', range(228), size=sz
                                          ).normalize(imagenet_stats)

learn = cnn_learner(data, models.resnet34)
learn.load('stage-1');
learn.model.eval()
layer = learn.model._modules.get('1')._modules.get('5')

In [4]:
def get_vector(fpath):

    # img = Image.open(BytesIO(binary_data))
    img = Image.open(fpath)

    t_img = Variable(normalize(to_tensor(scaler(img)).cuda()).unsqueeze(0))
    my_embedding = torch.zeros((1, 512))

    def copy_data(m, i, o):
        my_embedding.copy_(o.data)

    h = layer.register_forward_hook(copy_data)
    learn.model(t_img)
    h.remove()

    return my_embedding.numpy()

In [5]:
emb_dict = {}

In [6]:
p = Path('../product_resized/')

In [7]:
for p in p.iterdir():
    if not str(p).endswith('.jpg'):
        pass
    if str(p).split('/')[-1][:-4] in emb_dict.keys():
        pass
    try:
        emb_dict[str(p).split('/')[-1][:-4]] = get_vector(p)[0]
    except Exception as e:
        print(e)

In [8]:
len(emb_dict)

30368

In [9]:
import pickle

In [10]:
pickle.dump(emb_dict, open("../models/feature_dict.prod", "wb"))

In [11]:
f = 512

In [12]:
t = AnnoyIndex(f)

In [14]:
for k,v in emb_dict.items():
    t.add_item(int(k), v)

In [15]:
t.build(100)

True

In [16]:
t.save('prod.ann')

True